<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_SE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai openai requests langchain_openai langchain deepseek-ai -q

In [3]:
import os
import time
import json
import re
import logging
import concurrent.futures
from openai import OpenAI
from google.colab import userdata
from pydantic import BaseModel
from typing import Dict, List, Any

# Configure logging for detailed traceability
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Custom exception for API key errors
class APIKeyError(Exception):
    """Raised when API key retrieval or validation fails."""
    pass

# --- Pydantic Models for Agent and Task ---

class Agent(BaseModel):
    role: str
    goal: str
    backstory: str
    tools: Dict[str, Any]

class Task(BaseModel):
    description: str
    expected_output: str
    input_key: str

# --- LLM and Tool Functions ---

def deepseek_inference(prompt: str, model: str, api_key: str) -> str:
    """
    Calls the DeepSeek API with the provided prompt and model using openai library.
    """
    try:
        client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com/v1")
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"DeepSeek API call failed: {e}")
        return f"Error: {str(e)}"

def code_executor_tool(code: str) -> str:
    """Executes Python code in a sandboxed environment and returns output."""
    try:
        local_vars = {}
        exec(code, {}, local_vars)
        return f"Code executed successfully. Variables: {local_vars}"
    except Exception as e:
        return f"Code execution failed: {str(e)}"

def github_issue_analyzer_tool(issue_url: str) -> str:
    """Analyzes a GitHub issue (placeholder; replace with real API call)."""
    logger.info(f"Analyzing GitHub issue: {issue_url}")
    return f"Analyzed issue {issue_url}: High priority bug in auth module."

def get_api_key() -> str:
    """Retrieves DeepSeek API key securely from Colab secrets or environment variables."""
    try:
        api_key = userdata.get('DEEPSEEK_API_KEY')
        if not api_key:
            api_key = os.getenv("DEEPSEEK_API_KEY")
        if not api_key:
            raise APIKeyError("No DEEPSEEK_API_KEY found in Colab secrets or environment variables.")
        return api_key
    except Exception as e:
        logger.error(f"Failed to retrieve API key: {e}")
        raise APIKeyError(f"API key retrieval error: {str(e)}")

# --- Agentic Logic and Orchestration ---

def create_agent_prompt(agent: Agent, tools_str: str, input_text: str, scratchpad: str) -> str:
    """
    Creates a structured prompt for the LLM based on the agent's persona and tools.
    """
    return f"""
You are a {agent.role} with the goal: {agent.goal}.
Backstory: {agent.backstory}

Available tools:
{tools_str}

Use the following format for your response:
Thought: you should always think about what to do
Action: the action to take, should be one of [{', '.join(agent.tools.keys())}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now have the final answer
Final Answer: the final answer to the original input question

Input: {input_text}
Scratchpad:
{scratchpad}
"""

def execute_agent_task(agent: Agent, task: Task, inputs: Dict[str, str], api_key: str) -> Dict[str, Any]:
    """
    Runs a single task with the agent's logic loop, replacing the LangChain AgentExecutor.
    """
    input_text = task.description.format(**inputs)
    scratchpad = ""
    max_steps = 10

    # Format tools for the prompt
    tools_str = "\n".join([f"- {name}: {tool_func.__doc__.strip()}" for name, tool_func in agent.tools.items()])

    try:
        for step in range(max_steps):
            prompt = create_agent_prompt(agent, tools_str, input_text, scratchpad)
            response = deepseek_inference(prompt, "deepseek-reasoner", api_key)
            logger.info(f"Agent thought process for step {step+1}:\n{response}")

            final_answer_match = re.search(r"Final Answer:\s*(.*)", response, re.DOTALL)
            if final_answer_match:
                result = final_answer_match.group(1).strip()
                return {"task": input_text, "result": result}

            action_match = re.search(r"Action:\s*(.*)\nAction Input:\s*(.*)", response, re.DOTALL)
            if action_match:
                action = action_match.group(1).strip()
                action_input = action_match.group(2).strip().strip('"')

                if action in agent.tools:
                    tool_func = agent.tools[action]
                    observation = tool_func(action_input)
                    scratchpad += f"\nThought: {response}\nAction: {action}\nAction Input: {action_input}\nObservation: {observation}\n"
                else:
                    observation = f"Error: Tool '{action}' not found."
                    scratchpad += f"\nThought: {response}\nError: {observation}\n"
            else:
                observation = "Error: Could not parse Action/Action Input from LLM response."
                scratchpad += f"\nThought: {response}\nError: {observation}\n"

            logger.info(f"Observation: {observation}")
            time.sleep(1)

        return {"task": input_text, "result": "Agent failed to find a final answer within the step limit."}

    except Exception as e:
        logger.error(f"Task execution failed for '{input_text}': {e}")
        return {"task": input_text, "result": f"Error: {str(e)}"}

# --- Main Execution ---

def main():
    """Main function to run the agentic software engineering system."""
    try:
        api_key = get_api_key()

        # Define all available tools
        available_tools = {
            "CodeExecutor": code_executor_tool,
            "GitHubIssueAnalyzer": github_issue_analyzer_tool
        }

        # Create agents with specific tools
        agents = [
            Agent(
                role="Bug Triage Specialist",
                goal="Triage and prioritize bugs from issue trackers",
                backstory="You analyze GitHub issues, assign priorities, and suggest fixes.",
                tools={"GitHubIssueAnalyzer": available_tools["GitHubIssueAnalyzer"]}
            ),
            Agent(
                role="Testing Engineer",
                goal="Generate and execute tests for code changes",
                backstory="You create unit and integration tests, validate fixes, and report coverage.",
                tools={"CodeExecutor": available_tools["CodeExecutor"]}
            ),
            Agent(
                role="Code Migration Expert",
                goal="Migrate code across versions or languages",
                backstory="You refactor code (e.g., Python 2 to 3) while ensuring compatibility.",
                tools={"CodeExecutor": available_tools["CodeExecutor"]}
            ),
            Agent(
                role="Legacy Modernizer",
                goal="Modernize outdated codebases",
                backstory="You refactor legacy code (e.g., COBOL to Java) using modern patterns.",
                tools={"CodeExecutor": available_tools["CodeExecutor"]}
            )
        ]

        # Define tasks
        tasks = [
            Task(description="Triage the following bug report: {bug_report}", expected_output="", input_key="bug_report"),
            Task(description="Generate and run tests for code: {code_snippet}", expected_output="", input_key="code_snippet"),
            Task(description="Migrate this Python 2 code to Python 3: {legacy_code}", expected_output="", input_key="legacy_code"),
            Task(description="Modernize this legacy COBOL snippet to Java: {legacy_code}", expected_output="", input_key="legacy_code")
        ]

        # Input dictionary
        inputs = {
            "bug_report": "User reports auth failure on login (GitHub issue #123).",
            "code_snippet": "def add(a, b): return a + b",
            "legacy_code": "print 'Hello World'"
        }

        # Execute tasks in parallel
        task_outputs = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            future_to_task = {
                executor.submit(execute_agent_task, agents[i], task, inputs, api_key): task
                for i, task in enumerate(tasks)
            }
            for future in concurrent.futures.as_completed(future_to_task):
                task_outputs.append(future.result())

        # Generate and log final report
        report = json.dumps(task_outputs, indent=4)
        logger.info("\n--- Final Report ---\n" + report)
        print("\n--- Final Report ---\n" + report)


    except APIKeyError as e:
        logger.error(f"API key error: {e}")
    except Exception as e:
        logger.error(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()


--- Final Report ---
[
    {
        "task": "Triage the following bug report: User reports auth failure on login (GitHub issue #123).",
        "result": "This is a **High Priority** bug. Authentication failures block user access and affect multiple users. The issue appears to be server-side (invalid credential errors despite correct credentials) and is reproducible. Immediate investigation into authentication service/logs is recommended. Check for recent deployments to staging that might have introduced this regression."
    },
    {
        "task": "Migrate this Python 2 code to Python 3: print 'Hello World'",
        "result": "The Python 2 code `print 'Hello World'` should be migrated to Python 3 as `print('Hello World')`. This change ensures compatibility with Python 3's print function syntax."
    },
    {
        "task": "Modernize this legacy COBOL snippet to Java: print 'Hello World'",
        "result": "The modernized Java code equivalent to the COBOL snippet is:\n\n```java